In [1]:
# Instalar Dash e JupyterDash
!pip install dash
!pip install jupyter-dash


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 15.6 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [2]:
# Bibliotecas
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import pandas as pd

In [3]:

# Carregar dados
# df_planilha1 = pd.read_csv('/content/3obj-sem-geracao-all-events(com2020)-nse.csv')
# df_planilha2 = pd.read_csv('/content/3obj-sem-geracao-all-events(com2020)-rmse.csv')
# df_planilha3 = pd.read_csv('/content/3obj-sem-geracao-all-events(com2020)-kge.csv')
# df_planilha4 = pd.read_csv('/content/5obj-sem-geracao-all_4-events.csv')

df_planilha1 = pd.read_csv('data/3obj-sem-geracao-all-events(com2020)-nse.csv')
df_planilha2 = pd.read_csv('data/3obj-sem-geracao-all-events(com2020)-rmse.csv')
df_planilha3 = pd.read_csv('data/3obj-sem-geracao-all-events(com2020)-kge.csv')
df_planilha4 = pd.read_csv('data/5obj-sem-geracao-all_4-events.csv')


planilhas = {
    'NSE, moran_mean, lisa_stdev': df_planilha1,
    'RMSE, moran_mean, lisa_stdev': df_planilha2,
    'KGE, moran_mean, lisa_stdev': df_planilha3,
    'NSE, RMSE, KGE, moran_mean, lisa_stdev': df_planilha4
}


def get_columns(df):
    return [{'label': col, 'value': col} for col in df.columns if col not in ['ID', 'front', 'front color', 'Unnamed: 0']]

# Inicializar o app Dash
app = Dash(__name__)

app.layout = html.Div([
    html.H2('Gráfico Multiobjetivo: Visualização interativa de cenários com três ou cinco objetivos'),

    html.H4("Selecione a combinação de métricas e as variáveis dos eixos:"),

    # Dropdown para selecionar a planilha
    html.Label('Selecione a combinação de métricas:'),
    dcc.Dropdown(
        id='planilha-dropdown',
        options=[{'label': key, 'value': key} for key in planilhas.keys()],
        value='NSE, moran_mean, lisa_stdev',  # Valor inicial
        clearable=False
    ),

    # Dropdowns para selecionar os eixos
    html.Label('Selecione a métrica do eixo X:'),
    dcc.Dropdown(id='x-axis', value='RMSE'),

    html.Label('Selecione a métrica do eixo Y:'),
    dcc.Dropdown(id='y-axis', value='lisa_stdev'),

    html.Label('Selecione a métrica do eixo Z:'),
    dcc.Dropdown(id='z-axis', value='moran_mean'),

    # Gráfico interativo
    dcc.Graph(id='scatter-plot')
])

# Callback para atualizar os eixos quando a planilha for alterada
@app.callback(
    [Output('x-axis', 'options'),
     Output('y-axis', 'options'),
     Output('z-axis', 'options'),
     Output('x-axis', 'value'),
     Output('y-axis', 'value'),
     Output('z-axis', 'value')],
    Input('planilha-dropdown', 'value')
)
def update_axes(planilha_selecionada):
    df = planilhas[planilha_selecionada]
    columns = get_columns(df)

    if len(columns) < 3:
        default_value = 'NSE'
        columns = [{'label': default_value, 'value': default_value}] * 3

    return columns, columns, columns, columns[0]['value'], columns[1]['value'], columns[2]['value']

# Callback para atualizar o gráfico
@app.callback(
    Output('scatter-plot', 'figure'),
    [Input('x-axis', 'value'),
     Input('y-axis', 'value'),
     Input('z-axis', 'value'),
     Input('planilha-dropdown', 'value')]
)
def update_graph(x_axis, y_axis, z_axis, planilha_selecionada):
    df = planilhas[planilha_selecionada]


    fig = px.scatter_3d(df, x=x_axis, y=y_axis, z=z_axis,
                        color='front',
                        hover_data={'ID': True},
                        color_continuous_scale='viridis_r')

    # Customizar hover
    fig.update_traces(
        marker=dict(showscale=True),
        hovertemplate='<br><b>ID:</b> %{customdata[0]}<br>' +
                      '<b>' + x_axis + ':</b> %{x}<br>' +
                      '<b>' + y_axis + ':</b> %{y}<br>' +
                      '<b>' + z_axis + ':</b> %{z}<extra></extra>'
    )

    fig.update_layout(coloraxis_showscale=True)

    return fig

if __name__ == '__main__':
    #app.run_server(debug=True)
    #app.run_server(mode='inline')   #visualização no próprio notebook Colab
    app.run(jupyter_mode="external") #criar um link para abrir a visualização em outra guia


Dash app running on:
Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>